In [159]:
import pandas as pd
import requests
import numpy as np
import json
from gvol import GVol
from plotly.subplots import make_subplots
from datetime import datetime
import plotly.graph_objects as go
import concurrent.futures
import datetime
import time
import threading

GVOL_API_KEY = "GVOL_API_KEY"
gvol_client = GVol(header='gvol-lite',gvol_api_key=GVOL_API_KEY)
url = "https://app.pinkswantrading.com/graphql"
pd.set_option('display.max_columns', None)

AD_COLORS = ['#1c3664', '#89bed8', '#f16623']

### Retrieve Available Options Expirations (Optional)

In [160]:
# url = "https://app.pinkswantrading.com/graphql"

# payload="{\"query\":\"query HifiVolSurfaceStrikesGreeksHourly(\\n  $symbol: BTCOrETHEnumType, \\n  $date: String, \\n $interval:String,\\n$exchange: ExchangeEnumType ){\\n  HifiVolSurfaceStrikesGreeksHourly(\\n    symbol: $symbol, \\n    date: $date, \\n    interval: $interval, \\n    exchange:$exchange) {\\n    date\\n    currency\\n    expiration\\n    strike\\n    instrumentName\\n    interval\\n    putCall\\n    spot\\n    underlyingPrice\\n    openInterest\\n    bidIv\\n    markIv\\n    askIv\\n    bestBidAmount\\n    bestBidPrice\\n    markPrice\\n    bestAskPrice\\n    bestAskAmount\\n    delta\\n    gamma\\n    theta\\n    vega\\n    rho\\n  } \\n}\\n\",\"variables\":{\"symbol\":\"ETH\",\"date\":\"2023-06-11\",\"interval\":\"1 hour\",\"exchange\":\"deribit\"}}"
# headers = {
#   'x-oracle': GVOL_API_KEY,
#   'Content-Type': 'application/json',
#   'accept': '*/*',
#   'Accept-Language': 'en-US,en;q=0.9'
# }

# response = requests.request("GET", url, headers=headers, data=payload)

# data = json.loads(response.text)

# chain = pd.json_normalize(data['data']['HifiVolSurfaceStrikesGreeksHourly'])

# chain['date'] = pd.to_datetime(chain['date'], unit='ms')
# chain['expiration'] = pd.to_datetime(chain['expiration'], unit='ms')
# chain['interval'] = pd.to_datetime(chain['interval'], unit='ms')

# # Get unique expiration dates and convert them to strings
# unique_expirations = chain['expiration'].dt.strftime('%Y-%m-%d').unique()

# print("Unique Expirations")

# for expiration in unique_expirations:
#     print(expiration)

### Initialize arguments for options data query

In [161]:
# Define ticker symbol
symbol = "BTC"

# Define the start and end dates
start_date = datetime.datetime(2023, 3, 1)
end_date = datetime.datetime(2023, 7, 25)

# Define the options and their positions
options_positions = {f"{symbol}-29MAR24-20000-C": "short", f"{symbol}-29MAR24-20000-P": "short"}

### Query Selected Options
Note: This may take a while to run since it runs many API calls simultaneously

In [162]:
# Define the headers
headers = {
    'x-oracle': GVOL_API_KEY,
    'Content-Type': 'application/json',
    'accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9'
}

# Create a Semaphore that allows 25 threads to run simultaneously
semaphore = threading.Semaphore(25)

def fetch_data_for_date(date):
    # Acquire the semaphore
    with semaphore:
        # Define the payload with the date for this iteration
        payload = {
            "query": "query HifiVolSurfaceStrikesGreeksHourly($symbol: BTCOrETHEnumType, $date: String, $interval:String,$exchange: ExchangeEnumType ){HifiVolSurfaceStrikesGreeksHourly(symbol: $symbol, date: $date, interval: $interval, exchange:$exchange) {date currency expiration strike instrumentName interval putCall spot underlyingPrice openInterest bidIv markIv askIv bestBidAmount bestBidPrice markPrice bestAskPrice bestAskAmount delta gamma theta vega rho } }",
            "variables": {"symbol": f"{symbol}", "date": date.strftime('%Y-%m-%d'), "interval": "1 hour", "exchange": "deribit"}
        }

        # Make the request
        response = requests.request("GET", url, headers=headers, data=json.dumps(payload))

        # Load the data
        data = json.loads(response.text)

        # Normalize the data
        chain = pd.json_normalize(data['data']['HifiVolSurfaceStrikesGreeksHourly'])

        # Convert the date, expiration, and interval to datetime objects
        chain['date'] = pd.to_datetime(chain['date'], unit='ms')
        chain['expiration'] = pd.to_datetime(chain['expiration'], unit='ms')
        chain['interval'] = pd.to_datetime(chain['interval'], unit='ms')

        # Filter the data for the options you're interested in
        chain = chain[chain['instrumentName'].isin(options_positions.keys())]

        # Add a column for the query date
        chain['query_date'] = date

        # Add a column for the position (long/short)
        chain['direction'] = chain['instrumentName'].map(options_positions)

        return chain

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # For each of the dates in the range
    dataframes = []
    for single_date in pd.date_range(start_date, end_date):
        # Start a new thread to fetch the data for this date and add the future to the list
        dataframes.append(executor.submit(fetch_data_for_date, single_date))

    # Replace the futures in the list with their results after they finish
    dataframes = [future.result() for future in concurrent.futures.as_completed(dataframes)]

# Concatenate all the dataframes into a single dataframe
data = pd.concat(dataframes)

# Sort the dataframe by the query date
data = data.sort_values('query_date')

data.head()

,date,currency,expiration,strike,instrumentName,interval,putCall,spot,underlyingPrice,openInterest,bidIv,markIv,askIv,bestBidAmount,bestBidPrice,markPrice,bestAskPrice,bestAskAmount,delta,gamma,theta,vega,rho,query_date,direction
18500,2023-03-30 23:00:00,BTC,2024-03-29 08:00:00,20000,BTC-29MAR24-20000-P,2023-03-30 23:00:00,P,27858.67,28662.77,1.6,59.66,62.10,64.32,21.0,0.0815,0.0880,0.094,9.0,-0.18667,0.00002,-6.55079,76.87214,-78.60162,2023-03-30,short
6160,2023-03-30 08:25:00,BTC,2024-03-29 08:00:00,20000,BTC-29MAR24-20000-C,2023-03-30 08:00:00,C,28692.18,29537.06,0.0,0.00,55.42,0.00,0.0,0.0000,0.3862,0.000,0.0,0.83663,0.00002,-5.53033,72.84821,133.02335,2023-03-30,short
6194,2023-03-30 08:25:00,BTC,2024-03-29 08:00:00,20000,BTC-29MAR24-20000-P,2023-03-30 08:00:00,P,28692.09,29537.26,0.0,19.81,55.42,0.00,10.0,0.0015,0.0634,0.000,0.0,-0.16337,0.00002,-5.53031,72.84783,-66.96587,2023-03-30,short
6978,2023-03-30 09:00:00,BTC,2024-03-29 08:00:00,20000,BTC-29MAR24-20000-C,2023-03-30 09:00:00,C,28584.62,29431.06,0.0,0.00,60.84,0.00,0.0,0.0000,0.3983,0.000,0.0,0.82619,0.00001,-6.29550,75.53373,125.91249,2023-03-30,short
7012,2023-03-30 09:00:00,BTC,2024-03-29 08:00:00,20000,BTC-29MAR24-20000-P,2023-03-30 09:00:00,P,28584.41,29431.35,0.5,57.94,60.83,61.66,11.7,0.0705,0.0779,0.080,0.1,-0.17380,0.00001,-6.29525,75.53270,-74.06158,2023-03-30,short


### Plot Time Series for Selective Options

In [163]:
def plot_time_series(data, column, options_positions):
    """
    This function creates a plot of the specified column as a time series.

    Parameters:
    data (DataFrame): The DataFrame containing the data to plot.
    column (str): The name of the column to plot.
    options_positions (dict): The dictionary containing the options and their positions.

    Returns:
    None. Displays a plot.
    """
    # Create a deep copy of 'data' to manipulate without adjusting the original dataframe
    options_data = data.copy(deep=True)

    # Add a column for the position (long/short)
    options_data['direction'] = options_data['instrumentName'].map(options_positions)

    # Locate values to change for short positions
    cols_to_change = ['bestBidPrice', 'markPrice', 'bestAskPrice',
                    'delta', 'gamma', 'theta', 'vega', 'rho']

    # Change short options positions to negative values
    for col in cols_to_change:
        options_data.loc[options_data['direction'] == 'short', col] *= -1

    # Check if the column exists in the data
    if column not in options_data.columns:
        print(f"Column {column} not found in the data.")
        return

    # Set the 'date' as the index and resample to hourly
    options_data.set_index('date', inplace=True)
    options_data = options_data.resample('H').sum(numeric_only=True)

    # Group by query_date and sum the values of the column
    aggregated_data = options_data.groupby(options_data.index)[column].sum().reset_index()

    # Create a figure
    fig = go.Figure()

    # Add a line trace for the aggregated data
    fig.add_trace(go.Scatter(
        x=aggregated_data['date'],
        y=aggregated_data[column],
        mode='lines',
        name=column,
        line=dict(color='black')
    ))

    # Set layout properties
    options_expirations = ', '.join([f"{key}({val})" for key, val in options_positions.items()])
    fig.update_layout(
        title=f'{column} for {options_expirations}',
        yaxis_title=f'{column}',
        template='plotly_white'
    )

    # Show the figure
    fig.show()


"""Call the function with required parameters"""
# Use any available columns from in `data` in the "column" argument
plot_time_series(data, 'markPrice', options_positions)

### Plot Greeks for Selective Options

In [164]:
def plot_greeks(data, options_positions):
    """
    This function creates a plot of specified columns as a time series in subplots.

    Parameters:
    data (DataFrame): The DataFrame containing the data to plot.
    options_positions (dict): The dictionary containing the options and their positions.

    Returns:
    None. Displays a plot.
    """
    # Create a deep copy of 'data' to manipulate without adjusting the original dataframe
    options_data = data.copy(deep=True)

    # Add a column for the position (long/short)
    options_data['direction'] = options_data['instrumentName'].map(options_positions)

    # Columns to adjust for short positions
    cols_to_change = ['bestBidPrice', 'markPrice', 'bestAskPrice',
                    'delta', 'gamma', 'theta', 'vega', 'rho']

    # Adjust the columns for short call positions
    for col in cols_to_change:
        options_data.loc[(options_data['direction'] == 'short') & (options_data['putCall'] == 'C'), col] *= -1

    # Adjust the columns for short put positions
    for col in cols_to_change:
        options_data.loc[(options_data['direction'] == 'short') & (options_data['putCall'] == 'P'), col] *= -1

    # Set the 'date' as the index and resample to hourly
    options_data.set_index('date', inplace=True)
    options_data = options_data.resample('H').sum(numeric_only=True)

    # Create subplots
    fig = make_subplots(rows=5, cols=1, subplot_titles=['Delta', 'Gamma', 'Theta', 'Vega', 'Rho'])

    # Add a line trace for each greek
    for i, column in enumerate(['delta', 'gamma', 'theta', 'vega', 'rho'], start=1):
        # Group by date and sum the values of the column
        aggregated_data = options_data.groupby(options_data.index)[column].sum().reset_index()
        
        fig.add_trace(go.Scatter(
            x=aggregated_data['date'],
            y=aggregated_data[column],
            mode='lines',
            name=column,
            line=dict(color='black')
        ), row=i, col=1)

    # Set layout properties
    options_expirations = ', '.join([f"{key}({val})" for key, val in options_positions.items()])
    fig.update_layout(
        title=f'Greeks for {options_expirations}',
        template='plotly_white',
        height=900
    )

    # Show the figure
    fig.show()


"""Call the function with required parameters"""
plot_greeks(data, options_positions)